## Local DuckGPT Cookbook

This notebook shows common recipes for working with the local data using DuckGPT

### Create the conversation

Create the conversation and load the database if needed 

In [ ]:
from duckgpt.local.conversation import duckdb_local_agent, get_duckgpt_local_conversation
from duckgpt.local.load_tables import load_local_tables

duckgpt_conversation = get_duckgpt_local_conversation()

LOAD_DATABASE = True
if LOAD_DATABASE:
    load_local_tables(duckdb_agent=duckdb_local_agent)

### Chat with DuckGPT

You can chat with a conversation using:
1. The `print_response()` function that prints the reponse
2. The `chat()` function that returns the response

In [ ]:
duckgpt_conversation.print_response("Which actor has the highest average movie rating?")

In [ ]:
rating_histogram_response = duckgpt_conversation.chat(
    "Give me a histogram of movies by rating, decide the best bucket size?", stream=False
)
print(rating_histogram_response)

### Run a DuckDb Query

You can run a DuckDb in 2 ways:
1. Using the DuckDb connection directly
2. Using the `run_query()` function of the duckdb_agent

In [ ]:
duckdb_local_agent.run_query("SELECT * FROM Movies LIMIT 5")

In [ ]:
duckdb_local_agent.connection.sql("SELECT * FROM Movies LIMIT 5").show()